In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import glob
import numpy as np
plt.rcParams['font.family'] = 'Malgun Gothic' # Malgun Gothic을 적용
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
df = pd.read_csv('bi_8.csv', usecols=[1,2,3,4,5,6,7], index_col=0)
df

In [ ]:
df.describe()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
scaler.fit(df)

In [ ]:
df_scaled = scaler.transform(df)
df_scaled.shape

In [ ]:
df_scaled

In [ ]:
df_scaled = df_scaled.transpose()

In [ ]:
df_scaled.shape

In [ ]:
df_scaled_df = pd.DataFrame({
    '22시승차총승객수' : df_scaled[0],
    '22시하차총승객수' : df_scaled[1],
    '23시승차총승객수' : df_scaled[2],
    '23시하차총승객수' : df_scaled[3],
    '승차승객합' : df_scaled[4],
    '하차승객합' : df_scaled[5]}, index = df.index
)

In [ ]:
df_scaled_df

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize = (10, 10))
plt.scatter(df_scaled_df['승차승객합'], df_scaled_df['하차승객합'], 
            c = (df_scaled_df['승차승객합'] + df_scaled_df['하차승객합']), s = 50)
plt.colorbar()
plt.xlabel('승차승객합')
plt.ylabel('하차승객합')
plt.show()

In [ ]:
df_scaled_df['popular'] = (df_scaled_df['승차승객합'] + df_scaled_df['하차승객합']) / 2
df_sort = df_scaled_df.sort_values(by = 'popular', ascending=False)
df_sort

In [ ]:
plt.figure(figsize = (10, 9))
plt.scatter(df_sort['승차승객합'], df_sort['하차승객합'], 
            c = (df_sort['승차승객합'] + df_sort['하차승객합']), s = 50)

for n in range(20):
    plt.text(df_sort['승차승객합'][n]*1.01, df_sort['하차승객합'][n]*(0.97), df_sort.index[n], fontsize = 11)

plt.colorbar()
plt.xlabel('승차승객합')
plt.ylabel('하차승객합')
plt.show()

In [ ]:
# standard scaler
from sklearn.preprocessing import StandardScaler
scaler2 = StandardScaler()

df = pd.read_csv('bi_8.csv', usecols=[1,2,3,4,5,6,7], index_col=0)
df

In [ ]:
scaler2.fit(df)

In [ ]:
df_scaled2 = scaler2.transform(df)
df_scaled2.shape

In [ ]:
type(df_scaled2)

In [ ]:
df_scaled2 = df_scaled2.transpose()

In [ ]:
df_scaled_df2 = pd.DataFrame({
    '22시승차총승객수' : df_scaled2[0],
    '22시하차총승객수' : df_scaled2[1],
    '23시승차총승객수' : df_scaled2[2],
    '23시하차총승객수' : df_scaled2[3],
    '승차승객합' : df_scaled2[4],
    '하차승객합' : df_scaled2[5]
}, index = df.index
)

In [ ]:
plt.figure(figsize = (10, 10))
plt.scatter(df_scaled_df2['승차승객합'], df_scaled_df2['하차승객합'], 
            c = (df_scaled_df2['승차승객합'] + df_scaled_df2['하차승객합']), s = 50)
plt.colorbar()
plt.xlabel('승차승객합')
plt.ylabel('하차승객합')
plt.show()

In [ ]:
df_scaled_df2['popular'] = (df_scaled_df2['승차승객합'] + df_scaled_df2['하차승객합']) / 2
df_sort2 = df_scaled_df2.sort_values(by = 'popular', ascending=False)
df_sort2

In [ ]:
plt.figure(figsize = (10, 9))
plt.scatter(df_sort2['승차승객합'], df_sort2['하차승객합'], 
            c = (df_sort2['승차승객합'] + df_sort2['하차승객합']), s = 50)

for n in range(20):
    plt.text(df_sort2['승차승객합'][n]*1.01, df_sort2['하차승객합'][n]*(0.97), df_sort2.index[n], fontsize = 11)

plt.colorbar()
plt.xlabel('승차승객합')
plt.ylabel('하차승객합')
plt.show()

In [ ]:
# 클러스터링 구현
from sklearn.cluster import KMeans

dX = np.array([list(df_sort['승차승객합']), list(df_sort['하차승객합'])])
dX = dX.transpose()


In [ ]:
dX.shape

In [ ]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(dX)

In [ ]:
cluster_li = (kmeans.predict(dX))

In [ ]:
color_li = ['red','orange','yellow','green']

plt.figure(figsize = (10, 9))
for i in range(350):
    plt.scatter(df_sort['승차승객합'][i], df_sort['하차승객합'][i], 
                c = color_li[cluster_li[i]], s = 50)

for n in range(20):
    plt.text(df_sort['승차승객합'][n]*1.01, df_sort['하차승객합'][n]*(0.97), df_sort.index[n], fontsize = 11)

plt.xlabel('승차승객합')
plt.ylabel('하차승객합')
plt.show()

In [ ]:
import folium
import json
from folium import plugins 

In [ ]:
geo_path = 'skorea-submunicipalities-2018-geo.json'
geo_str = json.load(open(geo_path, encoding = 'utf-8'))

map = folium.Map(location = [37.5502, 126.982], zoom_start = 10.5)
folium.GeoJson(geo_str).add_to(map)

map

In [ ]:
map = folium.Map(location = [37.5502, 126.982], zoom_start = 10.5)
# folium.GeoJson(geo_str).add_to(map)

plugins.Fullscreen(
    position='topright',
    title='Expand me',
    title_cancel='Exit me',
    force_separate_button=True
).add_to(map)

cp = folium.Choropleth(geo_data = geo_str, data = df_sort['popular'],
              columns = [df_sort.index, df_sort['popular']],
              fill_color = 'PuRd', key_on = 'properties.name', highlight=True).add_to(map)

folium.GeoJsonTooltip(['name']).add_to(cp.geojson)

map

In [ ]:
df_sort[df_sort.index == '청룡동']

In [ ]:
dong = pd.read_excel('../../Statistical Learning/dong.xlsx', usecols='H:L', index_col=4)
dong.head()

In [ ]:
dong = dong.drop(['lat', 'lng'], axis=1)

In [ ]:
df_group = dong.groupby(dong.index).sum()
df_group

In [ ]:
scaler = MinMaxScaler()
scaler.fit(df_group)

scaled_group = scaler.transform(df_group)

scaled_group = scaled_group.transpose()

df_scaled_group = pd.DataFrame({
    '승차승객합' : scaled_group[0],
    '하차승객합' : scaled_group[1]}, index = df_group.index
)

In [ ]:
df_scaled_group['popular'] = df_scaled_group['승차승객합'] + df_scaled_group['하차승객합']

In [ ]:
import math
log_li = []
for i in df_scaled_group['popular']:
    log_li.append(math.log10(i+1))
# len(log_li)
df_scaled_group['pop_log'] = log_li

In [ ]:
index_li = []
for i in df_scaled_group.index:
    index_li.append(i.replace('.', '·'))
index_li    

In [ ]:
df_scaled_group.index = index_li
df_scaled_group[df_scaled_group.index == '금호2·3가동']

In [ ]:
map = folium.Map(location = [37.5502, 126.982], zoom_start = 10.5)
# folium.GeoJson(geo_str).add_to(map)

plugins.Fullscreen(
    position='topright',
    title='Expand me',
    title_cancel='Exit me',
    force_separate_button=True
).add_to(map)

cp = folium.Choropleth(geo_data = geo_str, data = df_scaled_group['pop_log'],
              columns = [df_scaled_group.index, df_scaled_group['pop_log']],
              fill_color = 'PuRd', key_on = 'properties.name', highlight=True).add_to(map)

folium.GeoJsonTooltip(['name']).add_to(cp.geojson)

map

In [ ]:
df_scaled_group[df_scaled_group.index == '구로동']